In [1]:
import numpy as np
import pickle
import os

from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import Bunch
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D
import os.path
import fnmatch
import itertools
import functools
from keras.models import Model
import tensorflow as tf

Using TensorFlow backend.


In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
# Check GPU is using or not
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess =  tf.compat.v1.Session(config=config)

In [4]:
train_data_dir = 'data/train'
validation_data_dir = 'data/val'

In [5]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [6]:
EPOCHS = 100
INIT_LR = 1e-3
BS = 8
default_image_size = tuple((224, 224))
image_size = 0
width=299
height=299
depth=3

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(height, width),
    batch_size=BS,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(height, width),
    batch_size=BS,
    class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(height, width),
    batch_size=BS,
    class_mode='categorical',
    shuffle = False)

Found 2800 images belonging to 7 classes.
Found 700 images belonging to 7 classes.
Found 700 images belonging to 7 classes.


In [8]:
base_model = InceptionV3(weights=None, include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(7, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
for layer in base_model.layers:
    layer.trainable = True

In [10]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('best_model.h5', verbose=1, monitor='val_accuracy', save_best_only=True, mode='auto')

In [12]:
history = model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.n // BS,
    validation_steps=validation_generator.n // BS,
    epochs=EPOCHS, verbose=1,
    callbacks=[checkpoint],
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
350/350 [==============================] - ETA: 0s - loss: 2.0845 - accuracy: 0.1971
Epoch 00001: val_accuracy improved from -inf to 0.15517, saving model to best_model.h5
350/350 [==============================] - 57s 162ms/step - loss: 2.0845 - accuracy: 0.1971 - val_loss: 16.6761 - val_accuracy: 0.1552
Epoch 2/100
350/350 [==============================] - ETA: 0s - loss: 1.6944 - accuracy: 0.2982
Epoch 00002: val_accuracy improved from 0.15517 to 0.17098, saving model to best_model.h5
350/350 [==============================] - 56s 159ms/step - loss: 1.6944 - accuracy: 0.2982 - val_loss: 4.2713 - val_accuracy: 0.1710
Epoch 3/100
350/350 [==============================] - ETA: 0s - loss: 1.6062 - accuracy: 0.3371
Epoch 00003: val_accuracy improved from 0.17098 to 0.38937, saving model to best_model.h5
350/350 [==============================] - 55s 158ms/step - loss: 1.6062 - accuracy: 0.3371 - val

350/350 [==============================] - ETA: 0s - loss: 0.3777 - accuracy: 0.8832
Epoch 00027: val_accuracy improved from 0.85345 to 0.85920, saving model to best_model.h5
350/350 [==============================] - 54s 155ms/step - loss: 0.3777 - accuracy: 0.8832 - val_loss: 0.4621 - val_accuracy: 0.8592
Epoch 28/100
350/350 [==============================] - ETA: 0s - loss: 0.3656 - accuracy: 0.8921
Epoch 00028: val_accuracy improved from 0.85920 to 0.87356, saving model to best_model.h5
350/350 [==============================] - 55s 157ms/step - loss: 0.3656 - accuracy: 0.8921 - val_loss: 0.5319 - val_accuracy: 0.8736
Epoch 29/100
350/350 [==============================] - ETA: 0s - loss: 0.3377 - accuracy: 0.9011
Epoch 00029: val_accuracy did not improve from 0.87356
350/350 [==============================] - 54s 155ms/step - loss: 0.3377 - accuracy: 0.9011 - val_loss: 0.4720 - val_accuracy: 0.8534
Epoch 30/100
350/350 [==============================] - ETA: 0s - loss: 0.3238 - a

350/350 [==============================] - ETA: 0s - loss: 0.2235 - accuracy: 0.9475
Epoch 00055: val_accuracy did not improve from 0.91667
350/350 [==============================] - 54s 156ms/step - loss: 0.2235 - accuracy: 0.9475 - val_loss: 1.1994 - val_accuracy: 0.8089
Epoch 56/100
350/350 [==============================] - ETA: 0s - loss: 0.2164 - accuracy: 0.9504
Epoch 00056: val_accuracy did not improve from 0.91667
350/350 [==============================] - 53s 152ms/step - loss: 0.2164 - accuracy: 0.9504 - val_loss: 2.3618 - val_accuracy: 0.8721
Epoch 57/100
350/350 [==============================] - ETA: 0s - loss: 0.1954 - accuracy: 0.9500
Epoch 00057: val_accuracy did not improve from 0.91667
350/350 [==============================] - 54s 154ms/step - loss: 0.1954 - accuracy: 0.9500 - val_loss: 0.5929 - val_accuracy: 0.8779
Epoch 58/100
350/350 [==============================] - ETA: 0s - loss: 0.1974 - accuracy: 0.9568
Epoch 00058: val_accuracy did not improve from 0.91667

Epoch 84/100
350/350 [==============================] - ETA: 0s - loss: 0.1602 - accuracy: 0.9704
Epoch 00084: val_accuracy did not improve from 0.91667
350/350 [==============================] - 54s 155ms/step - loss: 0.1602 - accuracy: 0.9704 - val_loss: 0.9848 - val_accuracy: 0.8721
Epoch 85/100
350/350 [==============================] - ETA: 0s - loss: 0.1798 - accuracy: 0.9661
Epoch 00085: val_accuracy improved from 0.91667 to 0.91810, saving model to best_model.h5
350/350 [==============================] - 55s 157ms/step - loss: 0.1798 - accuracy: 0.9661 - val_loss: 1.4295 - val_accuracy: 0.9181
Epoch 86/100
350/350 [==============================] - ETA: 0s - loss: 0.1730 - accuracy: 0.9714
Epoch 00086: val_accuracy did not improve from 0.91810
350/350 [==============================] - 54s 154ms/step - loss: 0.1730 - accuracy: 0.9714 - val_loss: 2.7724 - val_accuracy: 0.8592
Epoch 87/100
350/350 [==============================] - ETA: 0s - loss: 0.1874 - accuracy: 0.9668
Epoch 

In [13]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate_generator(generator=test_generator, steps=test_generator.n // BS +1)
print(f"Test Accuracy: {scores[1]*100}")

[INFO] Calculating model accuracy
Instructions for updating:
Please use Model.evaluate, which supports generators.
Test Accuracy: 92.28571653366089


In [14]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(test_generator, steps=test_generator.n // BS + 1)

y_pred = np.argmax(Y_pred, axis=1)
print()

print('Confusion Matrix')
target_names = list(test_generator.class_indices.keys())
print(target_names)
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')

print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Instructions for updating:
Please use Model.predict, which supports generators.

Confusion Matrix
['BulBul', 'Chorui', 'Crow', 'Doel', 'Machranga', 'Shalik', 'cuckoo']
[[87  8  3  0  0  1  1]
 [ 1 82 10  0  0  3  4]
 [ 0  0 87  6  0  1  6]
 [ 0  0  0 91  8  0  1]
 [ 0  0  3  0 89  5  3]
 [ 2  1  1  1  2 82 11]
 [ 8  2  7  2  0  0 81]]
Classification Report
              precision    recall  f1-score   support

      BulBul       0.89      0.87      0.88       100
      Chorui       0.88      0.82      0.85       100
        Crow       0.78      0.87      0.82       100
        Doel       0.91      0.91      0.91       100
   Machranga       0.90      0.89      0.89       100
      Shalik       0.89      0.82      0.85       100
      cuckoo       0.76      0.81      0.78       100

    accuracy                           0.86       700
   macro avg       0.86      0.86      0.86       700
weighted avg       0.86      0.86      0.86       700

